In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 读取数据
data = pd.read_csv('data.csv')  # 请根据实际情况修改文件路径和列名
print(data.head())

# 假设数据在第一列
time_series = data['your_column_name'].values.reshape(-1, 1)  # 请根据实际列名修改

# 归一化数据，将其缩放到0到1之间
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# 创建数据集：使用过去50个时间步的数据来预测下一个时间步
def create_dataset(data, time_step=50):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# 生成数据集
X, y = create_dataset(scaled_data, time_step=50)

# 重塑X以适应LSTM输入格式 [样本数, 时间步, 特征数]
X = X.reshape(X.shape[0], X.shape[1], 1)

# 构建多层LSTM模型
model = Sequential()

# 第一层 LSTM，返回每个时间步的输出以便传递给下一层
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))

# 第二层 LSTM，依然返回每个时间步的输出
model.add(LSTM(units=50, return_sequences=True))

# 第三层 LSTM，不需要返回每个时间步的输出，最后的输出用于回归或预测
model.add(LSTM(units=50))

# 输出层，预测下一个时间步的值
model.add(Dense(units=1))

# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error')

# 训练模型
model.fit(X, y, epochs=10, batch_size=32)

# 使用训练好的模型进行预测
predictions = model.predict(X)

# 将预测结果从0-1范围还原为原始值
predictions = scaler.inverse_transform(predictions)
y_actual = scaler.inverse_transform(y.reshape(-1, 1))

# 可视化预测结果
plt.plot(y_actual, color='blue', label='Actual Data')
plt.plot(predictions, color='red', label='Predicted Data')
plt.title('LSTM Model Prediction')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()
